# PlasticParcels tutorial for the DRAKKAR Workshop 2024 in Grenoble

<div class="alert alert-info">

The first few cells of this tutorial are pre-processing steps in order to get everything working on google colab.

</div>

### Install parcels and data download

In [ ]:
!mkdir data_download
!mkdir data
!mkdir PlasticParcels
!mkdir output_data

In [ ]:
## Install parcels
import zipfile
import os

In [ ]:
packages = ['cgen',
            'netcdf4>=1.1.9',
            'numpy>=1.9.1',
            'psutil',
            'pymbolic',
            'scipy',
            'tqdm',
            'xarray>=0.10.8',
            'cftime>=1.3.1',
            'dask>=2.0',
            'zarr>=2.11.0',
            'matplotlib-base >=2.0.2',
            'cartopy',
            'parcels']

for package in packages:
  os.system('pip install "'+package+'"')
  print(package)

In [ ]:
## Download required data for the tutorial
files = ["coordinates.zip",
         "release.zip",
         "unbeaching.zip",
         "daily_U.zip",
         "daily_V.zip",
         "daily_W.zip",
         "ERA5.zip"]

for file in files:
  print(file)
  os.system(f"curl https://plasticadrift.science.uu.nl/PlasticParcels_tutorialdata/{file} > ./data_download/{file}")

## Download the python-files for PlasticParcels
pp_files = ["helper.py",
            "kernels.py",
            "utils.py"]

for file in pp_files:
  print(file)
  os.system(f"curl https://raw.githubusercontent.com/OceanParcels/PlasticParcels/main/src/DRAKKAR%20workshop%20code/PlasticParcels/{file} > ./PlasticParcels/{file}")

In [ ]:
## Unzip data and delete the .zip file
download_files = os.listdir('./data_download/')
for file in download_files:
  with zipfile.ZipFile('./data_download/'+file, 'r') as zip_ref:
    # Unzip each file
    zip_ref.extractall('./data/')

    # Delete .zip file
    os.remove('./data_download/'+file)
    print(file, 'unzipped')
    zip_ref.close()

# PlasticParcels Tutorial
In this tutorial we will introduce you to some of the basic Parcels functions, as well as utilise the PlasticParcels tool for simulating microplastic pollution. In order to limit the depands on data transfer and storage/memory, this tutorial only works in the Mediterranean sea. But note that in general, the [PlasticParcels package](https://github.com/OceanParcels/PlasticParcels) works globally.

By the end of this tutorial, you should be able to create a fieldset that contains NEMO hydrodynamic (and other) fields, create a particleset that represent marine microplastic pollution release locations, and apply physics parameterisations to simulate the dispersal of microplastics in the ocean.

We will start by importing a number of necessary packages.

In [ ]:
## Library imports
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Parcels and PlasticParcels imports
from parcels import ParticleFile
from PlasticParcels.helper import create_fieldset, create_particleset, create_kernel, load_default_settings, create_particleset_from_file, load_test_settings
#from utils import select_files, create_directory, getclosest_ij

## Plotting imports
import matplotlib.pyplot as plt
import cartopy.crs as ccrs



# Model settings
Using PlasticParcels, we will load the default settings for our sumulations, using the `load_default_settings` function.

This returns two dictionaries of parameters, one used for fieldset creation, the other for particleset creation.

You can update the parameters by updating the dictionaries as necessary.

The description of each parameter can be found in the [PlasticParcels/helper.py](https://github.com/OceanParcels/PlasticParcels/blob/main/src/DRAKKAR%20workshop%20code/PlasticParcels/helper.py) file.

In [ ]:
## Load the default model and particle settings
model_settings, particle_settings = load_default_settings()

In [ ]:
## View settings
model_settings

In [ ]:
## Update settings

# e.g. turn off Stokes Drift
#model_settings['stokes_f'] = False

# e.g. turn off windage
#model_settings['wind_f'] = False

# Create a Parcels fieldset
We now create the fieldset that contains all the required fields (ocean velocities, temperature and salinity, ocean biology, winds and waves, etc.).

We loaded the default settings above, which we will pass into the `create_fieldset` PlasticParcels function. This removes a lot of the complexity when creating fieldsets with a large number of underlying fields. This function returns a Parcels fieldset object.

In [ ]:
fieldset = create_fieldset(model_settings, particle_settings)

print("Time origin of data: ", fieldset.time_origin)

To list all the fields that are present we use `.get_fields()` method to return a list of the fields, which we can then utilise and analyse.

In [ ]:
for parcels_field in fieldset.get_fields():
    print(parcels_field.name)

## Plotting Parcels Fields
Parcels fields are not loaded into memory immediately. Rather, the fields are only loaded when they are required (deferred load). Often, the hydrodynamic data alone can be hundreds of GBs. To visualise these fields we can load in the first timestep of the data.

We do this using the `.computeTimeChunk()` method, which takes as input a model time to load. Once loaded, we plot the resulting fields using the `pcolormesh` matplotlib function. Below is a small snippet of code used to plot these fields.

**A key highlight here is that Parcels handles different spatial grids and temporal resolutions automatically, and can even use different interpolation methods on each.**


<div class="alert alert-info">
Note - The plots below are indicative only. Since the data is on a C-grid, and the plotting assumes we are passing T-points, there is a slight spatial shift to the data. This is only an issue with our basic plotting below, Parcels correctly interprets the grids.
</div>

In [ ]:
## Load data into memory
fieldset.computeTimeChunk(0,1)

In [ ]:
## Plotting function
def plot_field(lons, lats, data, title):
    fig = plt.figure()
    ax = plt.axes(projection=ccrs.PlateCarree())
    colors = ax.pcolormesh(lons, lats, data, transform=ccrs.PlateCarree())
    ax.coastlines()
    ax.gridlines(draw_labels=True)
    plt.title(title)
    plt.colorbar(colors)
    plt.show()


Note that some fields have no depth component (e.g. wind, waves), and so the shape of their data is different, and should be indexed differently.

In [ ]:
print("The usual shape order is (time, depth, latitude, longitude)")
print("Ocean U velocity data shape:", fieldset.U.data.shape)
print("Ocean V velocity data shape:", fieldset.V.data.shape)
print("Wind U velocity data shape:", fieldset.Wind_U.data.shape)
print("Wind V velocity data shape:", fieldset.Wind_U.data.shape)

Plot the ocean surface velocity fields

In [ ]:
## Plot velocity fields
plot_field(fieldset.U.grid.lon, fieldset.U.grid.lat, fieldset.U.data[0,0], "Surface velocity in i-direction (m/s)")
plot_field(fieldset.V.grid.lon, fieldset.V.grid.lat, fieldset.V.data[0,0], "Surface velocity in j-direction (m/s)")

<div class="alert alert-info">
Note: The U,V fields are on a curvilinear grid, and the velocities are in the i- and j-directions. Parcels automatically rotates these vectors into the zonal- and meridional-directions when performing interpolation.
</div>

Plot the 10m-wind velocity fields

In [ ]:
## Plot Wind velocity fields
plot_field(fieldset.Wind_U.grid.lon, fieldset.Wind_U.grid.lat, fieldset.Wind_U.data[0], "Zonal wind velocity (m/s)")
plot_field(fieldset.Wind_V.grid.lon, fieldset.Wind_V.grid.lat, fieldset.Wind_V.data[0], "Meridional wind velocity (m/s)")

# Parcels can also be used as a standalone field interpolator
There are two ways to access the data in each Parcels field object. Firstly, as above, accessing the data directly and using the grid indices directly.

Another way to access the field data is to pass the field a (time, depth, latitude, longitude) tuple, from which Parcels will interpolate the data for you.

<div class="alert alert-info">
Note: The U,V fields are on a curvilinear grid, and the velocities are in the i- and j-directions. The UV field is a Parcels VectorField which references the U and V fields, appropriately rotating and scaling the i/j-vectors into zonal- and meridional-directions. Other velocity fields (e.g. wind) need their units to be converted when loaded as a field.
</div>


In [ ]:
# Sample the U,V ocean velocities at the ocean surface somewhere in the Mediterranean
(plat, plon) = (35.25, 20.11) # Point of Interest: 35.25deg North, 20.11deg East

U_value = fieldset.U[0, 0, plat, plon]
V_value = fieldset.V[0, 0, plat, plon]
print("Surface velocities in the i-, j-directions:", U_value, V_value, " (degrees/s).")

U_value, V_value = fieldset.UV[0, 0, plat, plon]
print("Surface velocities in the zonal-, meridional-directions:", U_value, V_value, " (degrees/s)")

We can also convert the values back into units of m/s by using the applyConversion parameter

In [ ]:
U_value, V_value = fieldset.UV.eval(0, 0, plat, plon, applyConversion=False)
print("Surface velocities in the zonal-, meridional-directions:", U_value, V_value, " (m/s)")

In [ ]:
## Determine the wind velocity at the point of interest
Wind_U_value = fieldset.Wind_U[0, 0, plat, plon]
Wind_V_value = fieldset.Wind_V[0, 0, plat, plon]
print("Surface wind velocities: ", Wind_U_value, Wind_V_value, " (degrees/s).")

Wind_U_value = fieldset.Wind_U.eval(0, 0, plat, plon, applyConversion=False)
Wind_V_value = fieldset.Wind_V.eval(0, 0, plat, plon, applyConversion=False)
print("Surface wind velocities: ", Wind_U_value, Wind_V_value, " (m/s).")

# Particle Sets
We will now create two instances of particlesets which represent release locations for microplastic particles.

In the first instance, we will create a basic particleset using Parcels. Since our microplastic particles require additional variables (e.g. windage coefficients and settling velocities), we will still use the PlasticParcels helper functions.

In the second instance, we will use PlasticParcels to create more complicated particlesets that replicate likely microplastic release locations. These release locations are determined using a range of datasets.


## Creating a simple particle set
Parcels includes a number of methods to create particle sets.

The most commonly used is `ParticleSet.from_list()`, which allows you to pass through a list of longitudes, latitudes, depths, and times.

Below, we randomly place particles in a box at different depths, and given these particles a wind coefficient, density, and diameter.

In [ ]:
## Number of particles to release
n_particles = 100

## Randomly place particle release locations on a grid
lons = np.random.uniform(2, 7, n_particles)
lats = np.random.uniform(37, 42, n_particles)

## Create a dictionary of lists containing release location information
release_locations = {'lons': lons,
                     'lats': lats
}

## Define particle properties
wind_coefficient = 0.01 # Percent of wind speed to apply
plastic_density = 1020. # Particle density (kg/m^3)
plastic_diameter = 0.001 # Partice diameter (m)
## Particleset settings
particle_settings.update({'release_locations': release_locations, # A dictionary of lons, lats, depths (optional), and times (optional)
                          'windage_coefficient': wind_coefficient, # A float or Nx1 array of windage coefficients, values in [0,1] [percent of wind speed]
                          'particle_density': plastic_density, # A float or Nx1 array of particle densities [kg m-3]
                          'particle_diameter': plastic_diameter, # A float or Nx1 array of particle lengths [m]
                          'write_output_option': 'none' # Option to write variables. Options: 'none' (default), 'tracer' (saves tracer fields), 'dynamic' (saves all variables that change spatiotemporally), 'all' (saves all variables)
                           })

In [ ]:
## Create the particleset, and kernels required for the simulation
pset = create_particleset(fieldset, particle_settings)
kernels = create_kernel(fieldset, pset)

Let's take a look at the particle set

In [ ]:
print(pset)

These particle sets can be indexed by particle id

In [ ]:
print("Information saved for particle_id = 4:", pset[4])

Plot the particles, and list the kernels that will be used

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.scatter(pset.lon, pset.lat, s=2, transform=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
plt.title("Initial particle positions")
plt.show()

fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.scatter(pset.lon, pset.lat, s=2, transform=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
ax.set_xlim([np.min(pset.lon)-25, np.max(pset.lon)+25])
ax.set_ylim([np.min(pset.lat)-25, np.max(pset.lat)+25])
plt.title("Initial particle positions")
plt.show()

List the kernels that will be used in the particle advection.
For documentation on the kernels, see [PlasticParcels/kernels.py](https://github.com/OceanParcels/PlasticParcels/blob/main/src/DRAKKAR%20workshop%20code/PlasticParcels/kernels.py).

In [ ]:
kernels

We will now advect these particles. We first need to create a Parcels `ParticleFile` where we will store our output. To advect the particles, we simply use the `execute` method.

In [ ]:
## Define the runtime, the timestepping, and the output frequency of the simulation
runtime = particle_settings['runtime']
dt_timestep = particle_settings['dt_timestep']
dt_write = particle_settings['dt_write']

## Create the particle file where output will be stored
pfile = ParticleFile('simple_release.zarr', pset, outputdt=dt_write)

## Execute the simulation
pset.execute(kernels, runtime=runtime, dt=dt_timestep, output_file=pfile)


Plot the final locations of the particles

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.scatter(pset.lon, pset.lat, s=2, transform=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
plt.title("Final particle positions")
plt.show()

fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.scatter(pset.lon, pset.lat, s=2, transform=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
ax.set_xlim([np.min(pset.lon)-25, np.max(pset.lon)+25])
ax.set_ylim([np.min(pset.lat)-25, np.max(pset.lat)+25])
plt.title("Final particle positions")
plt.show()

<div class="alert alert-info">
To analyse the output, you can follow the <a href="https://docs.oceanparcels.org/en/latest/examples/tutorial_output.html">Working with Parcels output tutorial</a>.
</div>

## Creating a more likely plastic particle release map

### Plastics emitted on the coast
We use the mismanaged plastic waste estimates by country from [Jambeck et. al "Plastic waste inputs from land into the ocean", Science (2015)](https://www.science.org/doi/10.1126/science.1260352) combined with the [NASA GPW v4.11 global population dataset](https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-density-rev11/) and the [Natural Earth coastline vector dataset](https://www.naturalearthdata.com/) in order to create a likely 'coastal' particle release map.


In [ ]:
# File locations
coastal = './data/coastal_population_MPW_NEMO0083.csv'
rivers = './data/river_emissions_NEMO0083.csv'
fisheries = './data/agg_data_fisheries_info.csv'

# Read and print coastal dataset
plastic_release = pd.read_csv(coastal)
plastic_release

### Particles emitted at river outlets
We use the data from [Meijer et al "More than 1000 rivers account for 80% of global riverine plastic emissions into the ocean" Science Advances (2021)](https://www.science.org/doi/10.1126/sciadv.aaz5803) combined with the Natural Earth coastline vector dataset to create a likely 'river' particle release map.

In [ ]:
# Read and print coastal dataset
plastic_release = pd.read_csv(rivers)
plastic_release

### Particles emitted from fishing-related activity
Lastly, we use the [Global Fishing Watch public fishing effort dataset](https://globalfishingwatch.org/data-download/datasets/public-fishing-effort) combined with the Natural Earth coastline vector dataset to create a likely 'fisheries' particle release map.

In [ ]:
# Read and print coastal dataset
plastic_release = pd.read_csv(fisheries)
plastic_release

### Create a particle set with these release settings

In [ ]:
## First we need to remove the release locations we used in the basic example above
del(particle_settings['release_locations'])

In [ ]:
## Remove any settings previously applied
settings_to_remove = ['release_type', 'particle_density', 'particle_diameter', 'windage_coefficient',
                      'continent', 'subregion', 'region', 'subregion', 'country']

for setting in settings_to_remove:
    if setting in particle_settings.keys():
        del(particle_settings[setting])


## Update your new settings here
new_settings = {'release_type': 'coastal',
                #'continent': '',
                #'region': '',
                #'subregion': 'Southern Europe',
                'country': 'Greece',
                'windage_coefficient' : 0.01, # Percentage of wind to apply to particles
                'particle_diameter' : 0.001, # Plastic particle diameter (m)
                'particle_density' : 1020. # Plastic particle density (kg/m^3)
}
particle_settings.update(new_settings)

## Create the particleset
pset = create_particleset_from_file(fieldset, particle_settings)


In [ ]:
## Plot the particles initial positions
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.scatter(pset.lon, pset.lat, s=2, transform=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
plt.title("Initial particle positions")
plt.show()

In [ ]:
## Define the runtime, the timestepping, and the output frequency of the simulation
runtime = particle_settings['runtime']
dt_timestep = particle_settings['dt_timestep']
dt_write = particle_settings['dt_write']

## Create the particle file where output will be stored
pfile = ParticleFile('advanced_release.zarr', pset, outputdt=dt_write)

## Execute the simulation
pset.execute(kernels, runtime=runtime, dt=dt_timestep, output_file=pfile)

In [ ]:
## Plot the particles final positions
fig = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())
ax.scatter(pset.lon, pset.lat, s=2, transform=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True)
plt.title("Final particle positions")
plt.show()

# Particle Trajectory Analysis
You can now analyse your particle sets. Rather than re-invent the wheel, we point you to the [Working with Parcels output tutorial](https://docs.oceanparcels.org/en/latest/examples/tutorial_output.html), which explains how to read and analyse Parcels particle trajectory datasets.